# Create basic RAG application with Azure AI Search

This sample demonstrates how to create a basic RAG application with Azure AI Search. forked from [azure-search-vector-samples](https://github.com/Azure/azure-search-vector-samples/tree/main)

> ✨ **_Note_** <br>
> Please check the regional support for Azure AI Search before you get started - https://learn.microsoft.com/en-us/azure/search/search-region-support
> In order to use the Semantic Search feature, check your region availability and pricing tier. Make sure it is at least Standard S3.

Create an `.env` file based on the `sample.env` file. Copy the new `.env` file to the folder containing your notebook and update the variables.


### References
- https://github.com/Azure/azure-search-vector-samples/blob/main/demo-python/code/integrated-vectorization/azure-search-integrated-vectorization-sample.ipynb

In [1]:
import os
import json
import requests

from openai import AzureOpenAI
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.search.documents import SearchIndexingBufferedSender
from azure.search.documents.models import (
    VectorizedQuery,
    VectorizableTextQuery,
    VectorFilterMode,
)
from azure.search.documents.models import QueryType, QueryCaptionType, QueryAnswerType
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    HnswAlgorithmConfiguration,
    VectorSearch,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    ComplexField,
    SearchIndex,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters,
)

# Load environment variables
load_dotenv(override=True)


# Validate environment variables
def get_env_var(name, required=True):
    value = os.getenv(name)
    if required and not value:
        raise ValueError(f"Environment variable {name} is missing or empty.")
    return value


try:
    azure_ai_search_endpoint = os.getenv("AZURE_AI_SEARCH_ENDPOINT")
    azure_ai_search_admin_key = get_env_var("AZURE_AI_SEARCH_API_KEY", required=False)

    azure_ai_inference_endpoint = get_env_var("AZURE_AI_INFERENCE_ENDPOINT")
    azure_ai_inference_key = get_env_var("AZURE_AI_INFERENCE_KEY", required=False)

    azure_openai_endpoint = get_env_var("AZURE_OPENAI_ENDPOINT")
    azure_openai_key = get_env_var("AZURE_OPENAI_API_KEY")
    azure_openai_deployment_name = get_env_var("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    azure_openai_embedding_deployment_name = (
        get_env_var("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME", required=False)
        or "text-embedding-ada-002"
    )
    azure_openai_embedding_dimensions = int(
        get_env_var("AZURE_OPENAI_EMBEDDING_DIMENSIONS", required=False) or 1536
    )
    azure_openai_api_version = (
        get_env_var("AZURE_OPENAI_API_VERSION", required=False) or "2024-12-01-preview"
    )

    index_name = (
        get_env_var("AZURE_SEARCH_INDEX_NAME", required=False)
        or "hotel_quickstart_vector"
    )

    print("==== Environment Variables ====")
    print(f"azure_search_endpoint={azure_ai_search_endpoint}")
    
    print(f"azure_ai_inference_endpoint={azure_ai_inference_endpoint}")
    
    print(f"azure_openai_endpoint={azure_openai_endpoint}")
    
    print(f"azure_openai_deployment_name={azure_openai_deployment_name}")
    print(
        f"azure_openai_embedding_deployment_name={azure_openai_embedding_deployment_name}"
    )
    print(f"azure_openai_embedding_dimensions={azure_openai_embedding_dimensions}")
    print(f"index_name={index_name}")

    # Validate credentials
    if azure_ai_search_admin_key:
        search_credential = AzureKeyCredential(azure_ai_search_admin_key)
    else:
        search_credential = DefaultAzureCredential()
        # Check if DefaultAzureCredential works
        try:
            search_credential.get_token("https://management.azure.com/.default")
        except Exception as e:
            raise ValueError(
                "DefaultAzureCredential authentication failed. Ensure you are logged in using `az login`."
            ) from e

    print("All environment variables are valid.")

except ValueError as e:
    print(f"[ERROR] {e}")
    exit(1)

==== Environment Variables ====
azure_search_endpoint=https://rag-innovator-search-svc.search.windows.net
azure_ai_inference_endpoint=https://<your-ai-inference>.services.ai.azure.com/models
azure_openai_endpoint=https://aoai-services1.openai.azure.com/
azure_openai_deployment_name=gpt-4o-mini
azure_openai_embedding_deployment_name=text-embedding-3-small
azure_openai_embedding_dimensions=1536
index_name=hotels-sample-index
All environment variables are valid.


### Create vector index



In [2]:
index_client = SearchIndexClient(
    endpoint=azure_ai_search_endpoint, credential=search_credential
)

In [3]:
from azure.core.exceptions import ResourceNotFoundError

try:
    index_client.get_index(index_name)
    index_client.delete_index(index_name)
    print(f"Index '{index_name}' deleted.")
except ResourceNotFoundError:
    print(f"Index '{index_name}' does not exist, creating a new one.")

Index 'hotels-sample-index' does not exist, creating a new one.


In [4]:
# vector field - titleVector, contentVector
fields = [
    SimpleField(name="HotelId", type=SearchFieldDataType.String, key=True),
    SearchableField(name="HotelName", type=SearchFieldDataType.String, sortable=True),
    SearchableField(
        name="Description", type=SearchFieldDataType.String, analyzer_name="en.lucene"
    ),
    # https://learn.microsoft.com/en-us/azure/search/search-language-support
    SearchableField(
        name="Description_kr",
        type=SearchFieldDataType.String,
        analyzer_name="ko.microsoft",
    ),
    SearchableField(
        name="Description_fr",
        type=SearchFieldDataType.String,
        analyzer_name="fr.lucene",
    ),
    SearchableField(
        name="Category",
        type=SearchFieldDataType.String,
        facetable=True,
        filterable=True,
        sortable=True,
    ),
    SearchableField(
        name="Tags",
        collection=True,
        type=SearchFieldDataType.String,
        facetable=True,
        filterable=True,
    ),
    SimpleField(
        name="ParkingIncluded",
        type=SearchFieldDataType.Boolean,
        facetable=True,
        filterable=True,
        sortable=True,
    ),
    SimpleField(
        name="LastRenovationDate",
        type=SearchFieldDataType.DateTimeOffset,
        facetable=True,
        filterable=True,
        sortable=True,
    ),
    SimpleField(
        name="Rating",
        type=SearchFieldDataType.Double,
        facetable=True,
        filterable=True,
        sortable=True,
    ),
    ComplexField(
        name="Address",
        fields=[
            SearchableField(name="StreetAddress", type=SearchFieldDataType.String),
            SearchableField(
                name="City",
                type=SearchFieldDataType.String,
                facetable=True,
                filterable=True,
                sortable=True,
            ),
            SearchableField(
                name="StateProvince",
                type=SearchFieldDataType.String,
                facetable=True,
                filterable=True,
                sortable=True,
            ),
            SearchableField(
                name="PostalCode",
                type=SearchFieldDataType.String,
                facetable=True,
                filterable=True,
                sortable=True,
            ),
            SearchableField(
                name="Country",
                type=SearchFieldDataType.String,
                facetable=True,
                filterable=True,
                sortable=True,
            ),
        ],
    ),
    SimpleField(
        name="Location",
        type=SearchFieldDataType.GeographyPoint,
        filterable=True,
        sortable=True,
    ),
    ComplexField(
        name="Rooms",
        collection=True,
        fields=[
            SearchableField(
                name="Description",
                type=SearchFieldDataType.String,
                analyzer_name="en.lucene",
            ),
            SearchableField(
                name="Description_kr",
                type=SearchFieldDataType.String,
                analyzer_name="ko.microsoft",
            ),
            SearchableField(
                name="Description_fr",
                type=SearchFieldDataType.String,
                analyzer_name="fr.lucene",
            ),
            SearchableField(
                name="Type",
                type=SearchFieldDataType.String,
                facetable=True,
                filterable=True,
            ),
            SimpleField(
                name="BaseRate",
                type=SearchFieldDataType.Double,
                facetable=True,
                filterable=True,
            ),
            SearchableField(
                name="BedOptions",
                type=SearchFieldDataType.String,
                facetable=True,
                filterable=True,
            ),
            SimpleField(
                name="SleepsCount",
                type=SearchFieldDataType.Int32,
                facetable=True,
                filterable=True,
            ),
            SimpleField(
                name="SmokingAllowed",
                type=SearchFieldDataType.Boolean,
                facetable=True,
                filterable=True,
            ),
            SearchableField(
                name="Tags",
                type=SearchFieldDataType.String,
                collection=True,
                facetable=True,
                filterable=True,
            ),
        ],
    ),
    SearchField(
        name="hotelNameVector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=azure_openai_embedding_dimensions,
        vector_search_profile_name="myHnswProfile",
    ),
    SearchField(
        name="descriptionVector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=azure_openai_embedding_dimensions,
        vector_search_profile_name="myHnswProfile",
    ),
    SearchField(
        name="descriptionKOVector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=azure_openai_embedding_dimensions,
        vector_search_profile_name="myHnswProfile",
    ),
]


# Define the vector search configuration
vector_search = VectorSearch(
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
            vectorizer_name="myVectorizer",
        )
    ],
    algorithms=[HnswAlgorithmConfiguration(name="myHnsw")],
    vectorizers=[
        AzureOpenAIVectorizer(
            vectorizer_name="myVectorizer",
            kind="azureOpenAI",
            parameters=AzureOpenAIVectorizerParameters(
                resource_url=azure_openai_endpoint,
                deployment_name=azure_openai_embedding_deployment_name,
                model_name=azure_openai_embedding_deployment_name,
                api_key=azure_openai_key,
            ),
        )
    ],
)

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="HotelName"),
        keywords_fields=[SemanticField(field_name="Category")],
        content_fields=[SemanticField(field_name="Description")],
    ),
)

# Create the semantic search with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

# Create the search index
index = SearchIndex(
    name=index_name,
    fields=fields,
    vector_search=vector_search,
    semantic_search=semantic_search,
)
result = index_client.create_or_update_index(index)

print(f"{result.name} created")

hotels-sample-index created


### Create embedding

Reads the document to index, embeds certain fields (HotelName, Description), and indexes them.

In [5]:
if azure_openai_endpoint is None:
    raise ValueError("The Azure OpenAI endpoint is not set.")

openai_client = AzureOpenAI(
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key,
)

In [6]:
# read the hotels_data.json file and save it to the output directory after embedding.
# You don't need to run this code if you have already run it once.

INDEX_JSON_DATA = True

if INDEX_JSON_DATA:
    hotels_data_file_path = "../sample-docs/hotels_data_ko.json"

    with open(file=hotels_data_file_path, mode="r", encoding="utf-8-sig") as file:
        hotel_documents = json.load(file)["value"]

    # document embedding

    # HotelName, Description embedding
    hotel_name = [item["HotelName"] for item in hotel_documents]
    description = [item["Description"] for item in hotel_documents]
    description_kr = [item["Description_kr"] for item in hotel_documents]

    hotel_name_response = openai_client.embeddings.create(
        model=azure_openai_embedding_deployment_name, input=hotel_name
    )
    hotel_name_embeddings = [item.embedding for item in hotel_name_response.data]

    description_response = openai_client.embeddings.create(
        model=azure_openai_embedding_deployment_name, input=description
    )
    description_embeddings = [item.embedding for item in description_response.data]

    description_kr_response = openai_client.embeddings.create(
        model=azure_openai_embedding_deployment_name, input=description_kr
    )
    description_kr_embeddings = [
        item.embedding for item in description_kr_response.data
    ]

    for i, item in enumerate(hotel_documents):
        item["hotelNameVector"] = hotel_name_embeddings[i]
        item["descriptionVector"] = description_embeddings[i]
        item["descriptionKOVector"] = description_kr_embeddings[i]

    # save the result in docVectors.json
    output_path = os.path.join(".", "output", "docVectors.json")
    output_directory = os.path.dirname(output_path)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    with open(output_path, "w") as f:
        json.dump(hotel_documents, f)

In [7]:
### input vector index
output_path = os.path.join(".", "output", "docVectors.json")
output_directory = os.path.dirname(output_path)

if not os.path.exists(output_directory):
    os.makedirs(output_directory)
with open(output_path, "r") as file:
    documents = json.load(file)

if azure_ai_search_endpoint is None:
    raise ValueError("The Azure AI Search endpoint is not set.")

search_client = SearchClient(
    endpoint=azure_ai_search_endpoint,
    index_name=index_name,
    credential=search_credential,
)
result = search_client.upload_documents(documents)
print(f"document indexing: {len(documents)}")

document indexing: 50


### VectorizableQuery, VectorizableTextQuery

- [VectorizableQuery](https://learn.microsoft.com/ko-kr/python/api/azure-search-documents/azure.search.documents.models.vectorizedquery?view=azure-python): User directly inputs embedding vectors to perform a search. In other words, the user passes the pre-vectorized values ​​to VectorizedQuery using a pre-trained model such as OpenAI, Hugging Face, Sentence Transformers, etc.

In [8]:
import time

# vector Search
# query = "WiFi가 제공되는 전통적인 호텔"
query = "뉴욕에 중심부의의 WiFi가 제공되는 역사있고 전통적인 호텔"

start_time = time.time()

embedding = (
    openai_client.embeddings.create(
        input=query, model=azure_openai_embedding_deployment_name
    )
    .data[0]
    .embedding
)

# Search with embdding query on descriptionVector field to return top 3 closest items
vector_query = VectorizedQuery(
    vector=embedding, k_nearest_neighbors=3, fields="descriptionVector"
)

results = search_client.search(
    search_text=None,
    vector_queries=[vector_query],
    select=["HotelName", "Description_kr", "Category"],
)

print("========== Result with VectorizedQuery ==========")
for result in results:
    print(f"HotelName: {result['HotelName']}")
    print(f"Score: {result['@search.score']}")
    # print(f"Description: {result['Description']}")
    print(f"Description_kr: {result['Description_kr']}")
    print(f"Category: {result['Category']}\n")

end_time = time.time()
print(f"Time taken for VectorizedQuery search: {end_time - start_time} seconds")

========== Result with VectorizedQuery ==========


HotelName: Stay-Kay City Hotel
Score: 0.64250153
Description_kr: 이 고전적인 호텔은 완전히 리모델링되었으며 뉴욕의 중심부에 있는 주요 상업 거리의 이상적인 위치에 있습니다. 몇 분 거리에 타임스 스퀘어와 도시의 역사적인 중심지, 그리고 뉴욕을 미국에서 가장 매력적이고 세계적인 도시 중 하나로 만드는 다른 명소들이 있습니다.
Category: Boutique

HotelName: Treehouse Hotel
Score: 0.6366472
Description_kr: 활기찬 도심과 분주한 비즈니스 지구의 중심 가까이에 위치한 저희 호텔에서 따뜻한 환대를 경험해 보세요. 무료 WiFi, 지역 교통편, 그리고 우유와 쿠키를 즐기실 수 있습니다.
Category: Budget

HotelName: Friendly Motor Inn
Score: 0.61995727
Description_kr: 역사적인 명소, 지역 관광지, 도시 공원과 가까운 곳에 위치해 있습니다. 공항과 카지노까지 무료 셔틀 서비스가 제공됩니다. 무료 아침 식사와 WiFi가 제공됩니다.
Category: Budget

Time taken for VectorizedQuery search: 0.46840524673461914 seconds


- [VectorizedTextQuery](https://learn.microsoft.com/ko-kr/python/api/azure-search-documents/azure.search.documents.models.vectorizabletextquery?view=azure-python): If the user directly enters text, Azure AI Search internally embeds (vectorizes) it and performs vector search. 

In [9]:
start_time = time.time()

vector_query = VectorizableTextQuery(
    text=query, k_nearest_neighbors=3, fields="descriptionVector"
)

results = search_client.search(
    search_text=None,
    vector_queries=[vector_query],
    select=["HotelName", "Description_kr", "Category"],
)

print("========== Result with VectorizableTextQuery  ==========")
for result in results:
    print(f"HotelName: {result['HotelName']}")
    print(f"Score: {result['@search.score']}")
    print(f"Description: {result['Description_kr']}")
    print(f"Category: {result['Category']}\n")

end_time = time.time()
print(f"Time taken for VectorizableTextQuery search: {end_time - start_time} seconds")

========== Result with VectorizableTextQuery  ==========


HotelName: Stay-Kay City Hotel
Score: 0.64250153
Description: 이 고전적인 호텔은 완전히 리모델링되었으며 뉴욕의 중심부에 있는 주요 상업 거리의 이상적인 위치에 있습니다. 몇 분 거리에 타임스 스퀘어와 도시의 역사적인 중심지, 그리고 뉴욕을 미국에서 가장 매력적이고 세계적인 도시 중 하나로 만드는 다른 명소들이 있습니다.
Category: Boutique

HotelName: Treehouse Hotel
Score: 0.6366472
Description: 활기찬 도심과 분주한 비즈니스 지구의 중심 가까이에 위치한 저희 호텔에서 따뜻한 환대를 경험해 보세요. 무료 WiFi, 지역 교통편, 그리고 우유와 쿠키를 즐기실 수 있습니다.
Category: Budget

HotelName: Friendly Motor Inn
Score: 0.61995727
Description: 역사적인 명소, 지역 관광지, 도시 공원과 가까운 곳에 위치해 있습니다. 공항과 카지노까지 무료 셔틀 서비스가 제공됩니다. 무료 아침 식사와 WiFi가 제공됩니다.
Category: Budget

Time taken for VectorizableTextQuery search: 1.5511412620544434 seconds


### Multi-Vector Search
- Cross-field vector search, which allows you to pass multiple query vectors to query multiple vector fields simultaneously.
- In this case, you can pass query vectors from two different embedding models to the corresponding vector fields in the index.
- For each vector field, you can also give different search settings, such as performing a vector search, weights, exhaustive KNN, etc.

In [10]:
query = "traditional hotels with free wifi"

vector_query_1 = VectorizableTextQuery(
    text=query,
    k_nearest_neighbors=3,
    fields="hotelNameVector",
    weight=1,
    exhaustive=True,
)
vector_query_2 = VectorizableTextQuery(
    text=query, k_nearest_neighbors=3, fields="descriptionVector", weight=0.7
)

results = search_client.search(
    search_text=None,
    vector_queries=[vector_query_1, vector_query_2],
    select=["HotelName", "Description", "Category"],
)

for result in results:
    print(f"HotelName: {result['HotelName']}")
    print(f"Score: {result['@search.score']}")
    print(f"Description: {result['Description']}")
    print(f"Category: {result['Category']}\n")

HotelName: Good Business Hotel
Score: 0.01666666753590107
Description: 1 Mile from the airport. Free WiFi, Outdoor Pool, Complimentary Airport Shuttle, 6 miles from Lake Lanier & 10 miles from downtown. Our business center includes printers, a copy machine, fax, and a work area.
Category: Suite

HotelName: Treehouse Hotel
Score: 0.016393441706895828
Description: Near the beating heart of our vibrant downtown and bustling business district. Experience the warmth of our hotel. Enjoy free WiFi, local transportation and Milk & Cookies.
Category: Budget

HotelName: Twin Vortex Hotel
Score: 0.016129031777381897
Description: New experience in the making. Be the first to experience the luxury of the Twin Vortex. Reserve one of our newly-renovated guest rooms today.
Category: Luxury

HotelName: Countryside Hotel
Score: 0.011666666716337204
Description: Save up to 50% off traditional hotels. Free WiFi, great location near downtown, full kitchen, washer & dryer, 24/7 support, bowling alley, fitne

### Using filters in Vector search

- Shows how to apply filters to your search.
- You can choose whether to use pre-filtering (the default) or post-filtering.

In [11]:
query = "traditional hotels with free wifi"

vector_query = VectorizableTextQuery(
    text=query, k_nearest_neighbors=3, fields="descriptionVector"
)

results = search_client.search(
    search_text=None,
    vector_queries=[vector_query],
    vector_filter_mode=VectorFilterMode.PRE_FILTER,
    filter="Category eq 'Budget'",
    select=["HotelName", "Description", "Category"],
)

for result in results:
    print(f"HotelName: {result['HotelName']}")
    print(f"Score: {result['@search.score']}")
    print(f"Description: {result['Description']}")
    print(f"Category: {result['Category']}\n")

HotelName: Friendly Motor Inn
Score: 0.6590744
Description: Close to historic sites, local attractions, and urban parks. Free Shuttle to the airport and casinos. Free breakfast and WiFi.
Category: Budget

HotelName: Thunderbird Motel
Score: 0.6488486
Description: Book Now & Save. Clean, Comfortable rooms at the lowest price. Enjoy complimentary coffee and tea in common areas.
Category: Budget

HotelName: Treehouse Hotel
Score: 0.6311266
Description: Near the beating heart of our vibrant downtown and bustling business district. Experience the warmth of our hotel. Enjoy free WiFi, local transportation and Milk & Cookies.
Category: Budget



### Hybrid search
- Performs a combination of Lexical and Vector searches and returns results.
- In the case of Vector search, you can improve the quality of your search by performing a lexical exact search along with a search using similarity.

In [12]:
query = "near downtown hotels"

vector_query = VectorizableTextQuery(
    text=query, k_nearest_neighbors=3, fields="descriptionVector"
)

# 어휘 검색(search_text=query)과 벡터 검색(vector_queries=[vector_query])을 함께 사용하여 검색 합니다.
results = search_client.search(
    search_text=query,
    vector_queries=[vector_query],
    select=["HotelName", "Description", "Category"],
    top=3,
)

for result in results:
    print(f"HotelName: {result['HotelName']}")
    print(f"Score: {result['@search.score']}")
    print(f"Description: {result['Description']}")
    print(f"Category: {result['Category']}\n")

HotelName: Luxury Lion Resort
Score: 0.03131881356239319
Description: Unmatched Luxury. Visit our downtown hotel to indulge in luxury accommodations. Moments from the stadium and transportation hubs, we feature the best in convenience and comfort.
Category: Luxury

HotelName: Foot Happy Suites
Score: 0.029462365433573723
Description: Downtown in the heart of the business district. Close to everything. Leave your car behind and walk to the park, shopping, and restaurants. Or grab one of our bikes and take your explorations a little further.
Category: Suite

HotelName: Smile Up Hotel
Score: 0.02916666865348816
Description: Experience the fresh, modern downtown. Enjoy updated rooms, bold style & prime location. Don't miss our weekend live music series featuring who's new/next on the scene.
Category: Suite



In [13]:
# 위의 검색 결과와, 벡터 검색만 사용(search_text=None) 의 결과를 비교해 보면, 다르다는 것을 알 수 있습니다.
vector_query = VectorizableTextQuery(
    text=query, k_nearest_neighbors=3, fields="descriptionVector"
)

results = search_client.search(
    search_text=None,
    vector_queries=[vector_query],
    select=["HotelName", "Description", "Category"],
    top=3,
)

for result in results:
    print(f"HotelName: {result['HotelName']}")
    print(f"Score: {result['@search.score']}")
    print(f"Description: {result['Description']}")
    print(f"Category: {result['Category']}\n")

HotelName: Smile Up Hotel
Score: 0.69187903
Description: Experience the fresh, modern downtown. Enjoy updated rooms, bold style & prime location. Don't miss our weekend live music series featuring who's new/next on the scene.
Category: Suite

HotelName: Luxury Lion Resort
Score: 0.6909753
Description: Unmatched Luxury. Visit our downtown hotel to indulge in luxury accommodations. Moments from the stadium and transportation hubs, we feature the best in convenience and comfort.
Category: Luxury

HotelName: Foot Happy Suites
Score: 0.6904679
Description: Downtown in the heart of the business district. Close to everything. Leave your car behind and walk to the park, shopping, and restaurants. Or grab one of our bikes and take your explorations a little further.
Category: Suite



### Semantic Hybrid Search

Hybrid search returns results that are captioned using the semantic ranker.

When creating the vector index, we added a sementic configuration and set the title, content, and keyword fields to use for semantic ranking, captions, highlighting, and answers.

``` python
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="HotelName"),
        keywords_fields=[SemanticField(field_name="Category")],
        content_fields=[SemanticField(field_name="Description")]
    )
)
```



In [14]:
query = "Good hotels for times square"

vector_query = VectorizableTextQuery(
    text=query, k_nearest_neighbors=3, fields="descriptionVector", exhaustive=True
)

# Hybrid & Semantic Search
results = search_client.search(
    search_text=query,
    vector_queries=[vector_query],
    select=["HotelName", "Description", "Category"],
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name="my-semantic-config",
    query_caption=QueryCaptionType.EXTRACTIVE,
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=3,
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"HotelName: {result['HotelName']}")
    print(f"Reranker Score: {result['@search.reranker_score']}")
    print(f"Description: {result['Description']}")
    print(f"Category: {result['Category']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

Semantic Answer: This<em> classic hotel </em>is<em> fully-refurbished and ideally located on the main commercial artery of the city in the heart of New York.</em> A few minutes away is Times Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities.
Semantic Answer Score: 0.9679999947547913

HotelName: Stay-Kay City Hotel
Reranker Score: 2.817405939102173
Description: This classic hotel is fully-refurbished and ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Times Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities.
Category: Boutique
Caption: This<em> classic hotel </em>is<em> fully-refurbished and ideally located on the main commercial artery of the city in the heart of New York.</em> A few minutes away is Times Square and the h